# Risk Parity Portfolio
***

In [20]:
import pandas as pd 
import numpy as np 
import yfinance as yf
from scipy.optimize import minimize #Imports the minimize function 

In [5]:
msft = yf.download(tickers = 'MSFT', period = '1mo')
lly = yf.download(tickers = 'LLY', period = '1mo')
cvx = yf.download(tickers = 'CVX', period = '1mo')
aapl = yf.download(tickers = 'AAPL', period = '1mo')
bhp = yf.download(tickers = 'BHP', period = '1mo')

/tmp/ipykernel_18205/1785097103.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  msft = yf.download(tickers = 'MSFT', period = '1mo')
[*********************100%***********************]  1 of 1 completed
/tmp/ipykernel_18205/1785097103.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  lly = yf.download(tickers = 'LLY', period = '1mo')
[*********************100%***********************]  1 of 1 completed
/tmp/ipykernel_18205/1785097103.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  cvx = yf.download(tickers = 'CVX', period = '1mo')
[*********************100%***********************]  1 of 1 completed
/tmp/ipykernel_18205/1785097103.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  aapl = yf.download(tickers = 'AAPL', period = '1mo')
[*********************100%***********************]  1 of 1 completed
/tmp/ipykernel_18205/1785097103.py:5: Future

In [10]:
print(msft.columns)

MultiIndex([( 'Close', 'MSFT'),
            (  'High', 'MSFT'),
            (   'Low', 'MSFT'),
            (  'Open', 'MSFT'),
            ('Volume', 'MSFT')],
           names=['Price', 'Ticker'])


In [ ]:
msft['Return'] = msft['Close'].pct_change()  #'why Adj close'
lly['Return'] = lly['Close'].pct_change()
cvx['Return'] = cvx['Close'].pct_change()
aapl['Return'] = aapl['Close'].pct_change()
bhp['Return'] = bhp['Close'].pct_change()

returns = pd.DataFrame({
    'MSFT':msft['Return'],
    'LLY':lly['Return'],
    'CVX':cvx['Return'],
    'AAPL':aapl['Return'],
    'BHP':bhp['Return']

})


#drop any rows that have Nan values
returns.dropna(inplace=True)

#Calculate the variance-covariance matrix
C = returns.cov()
C


,MSFT,LLY,CVX,AAPL,BHP
MSFT,0.000072,0.000081,0.000026,0.000041,0.000065
LLY,0.000081,0.000670,-0.000042,-0.000010,-0.000061
CVX,0.000026,-0.000042,0.000113,0.000080,0.000070
AAPL,0.000041,-0.000010,0.000080,0.000216,0.000163
BHP,0.000065,-0.000061,0.000070,0.000163,0.000389


# MCR = marginal contribution to Risk

In portfolio theory , MCR measures how much the total portfolio risk would change if you slightly increase weight of one asset

# RC = Risk Contribution

total risk from asset i

# In a risk parity portfolio 
A risk parity portfolio aims to equalize each assets contribution to total portfolio risk 

the optimization problem minimize the difference between each assets actual risk contirbution and the target (usually equal)


In [ ]:
#Method 1 - Implementation 
#SLSQP -> sequential least square programming

w = np.ones(5)/5
C = np.array(C)
def obj_function (w , C):
    p = C @ w
    MCR = p/np.sqrt(w.T @ C @ w)
    f = 0.0
    for i in range(len(w)):
        for j in range(len(w)):
            f+=(w[i]*MCR[i]-w[j]*MCR[j])**2
    return f 
    
def objective(w):
    return obj_function(w, C)

#constraints

constraints = (
    {'type':'eq', 'fun': lambda w:np.sum(w)-1}, #equality constraint sum(w)=1 
    {'type':'ineq', 'fun': lambda w:w}    #inequality constraint: w>= 0 
)

#bounds
bounds = [(0,1) for _ in range(C.shape[1])]

#optimize 
results = minimize(objective, w,method= 'SLSQP',constraints= constraints ,bounds = bounds,options={'disp':True})

#optimal weights
optimal_weights = results.x

print('optimal weights:',optimal_weights)
print('objective function value at optimal weights:',result.fun)  #closer to zero means risk contribution are nearly equal 
print('success:', result.success)
print('Message:', result.message)    

Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.3783292605971322e-05
            Iterations: 1
            Function evaluations: 6
            Gradient evaluations: 1
optimal weights: [0.2 0.2 0.2 0.2 0.2]
objective function value at iptumal weights: 2.3783292605971322e-05
success: True
Message: Optimization terminated successfully


In [22]:
#Method 2 - Implementation 
w = np.ones(5)/5 #initial guess for weight matrix
C = np.array(C)

def obj_function1(w, C):
    return np.sqrt(w.T @ C @ w)

def objective1(w):
    return obj_function(w, C)

#constraints

constraints =(
    {'type':'ineq','fun':lambda w:np.sum(np.log(w))+ 2},
    {'type':'ineq','fun':lambda w:w }
)

#bounds
bounds = [(0,1) for _ in range(C.shape[1])]

#optimize
result= minimize(objective, w, method='SLSQP',constraints=constraints,bounds=bounds,options={'disp':True})

# optimal weights
optimal_weights = result.x

print('optimal weights:',optimal_weights)
print('objective function value at optimal weights:',result.fun)  #closer to zero means risk contribution are nearly equal 
print('success:', result.success)
print('Message:', result.message)  

Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.786367541063129e-07
            Iterations: 26
            Function evaluations: 157
            Gradient evaluations: 26
optimal weights: [0.94634287 0.52434475 0.96866008 0.58117961 0.48453174]
objective function value at optimal weights: 1.786367541063129e-07
success: True
Message: Optimization terminated successfully
